# Notebook App for Agentic Research Assistant Agent Using Tavily, LangGrpah and Cohere Command R+ model

## Requirements

In [ ]:
!pip install langchain_cohere langchain-core langgraph langchain_core python-dotenv

## Libraries

In [2]:
import os
from typing import TypedDict, List, Annotated, Literal
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_cohere.chat_models import ChatCohere
from langgraph.graph import StateGraph, START, END, add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from tavily import AsyncTavilyClient, TavilyClient

/Users/maitarasher/anaconda3/envs/tavily/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


## Set API KEYS

In [3]:
# Set Your API Keys
TAVILY_API_KEY = "YOUR TAIVLY API KEY"
COHERE_API_KEY = "YOUR COHERE API KEY"

# Or use .env file 
from dotenv import load_dotenv
load_dotenv('.env')

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

In [4]:
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langchain_core.pydantic_v1 import BaseModel, Field


class ResearchState(TypedDict):
    user_query: str
    critique: str
    answer: str
    documents: List[dict]
    web_queries: List[str]
    revision_number: int
    max_revisions: int
    messages: Annotated[list[AnyMessage], add_messages]

# class SearchInput(BaseModel):
#     query: str = Field(description="should be a search query")
#     topic: str = Field(description="type of search, should be 'general' or 'news'")

# @tool("tavily_search",args_schema=SearchInput, return_direct=True)
# async def tavily_search(query: str, topic: str):
#     """Perform web search using the Tavily search tool."""
#     return await tavily_client.search(query=query, topic=topic)

#####
class SearchInput(BaseModel):
    sub_queries: List[str] = Field(description="break down the user's input into a set of sub-queries / sub-problems that can be answered in isolation")
    topic: str = Field(description="type of search, should be 'general' or 'news'")

@tool("tavily_search",args_schema=SearchInput, return_direct=True)
async def tavily_search(sub_queries: List[str], topic: str):
    """Perform searches for each sub-query using the Tavily search tool."""
    search_results = []
    for sub_query in sub_queries:
        results = await tavily_client.search(query=sub_query, topic=topic)
        print(results)
        search_results.append(results)
    print("search_results",search_results)
    return search_results


tools = [tavily_search]
# tool_node = ToolNode(tools)

tavily_client = AsyncTavilyClient(api_key=TAVILY_API_KEY)
model = ChatCohere(model="command-r-plus", temperature=0).bind_tools(tools)


tools_by_name = {tool.name: tool for tool in tools}
async def tool_node(state: dict):
    result = []
    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        print(tool)
        observation = await tool.ainvoke(tool_call["args"])
        print(observation)
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    return {"messages": result, "documents": result}
    
        
def call_model(state: ResearchState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define the function that determines whether to continue or not
def should_continue(state: ResearchState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END

# Define a graph
workflow = StateGraph(ResearchState)

# Add nodes
workflow.add_node("route_query", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as route_query
workflow.set_entry_point("route_query")

# Determine which node is called next
workflow.add_conditional_edges(
    "route_query",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# Add a normal edge from `tools` to `route_query`.
# This means that after `tools` is called, `route_query` node is called next.
workflow.add_edge("tools", "route_query")

app = workflow.compile()

In [5]:
messages = [
    HumanMessage(
        content="Wild fire prevention startups, divided by the type of technology"
    )
]

async for s in app.astream({"messages": messages}, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

# # Use the Runnable
# final_state = app.invoke(
#     {"messages": messages}
# )
# final_state["messages"][-1].content

# for event in app.stream({"messages": messages}):
#     for v in event.values():
#         print(v)

================================ Human Message =================================

Wild fire prevention startups, divided by the type of technology
================================== Ai Message ==================================

I will search for wildfire prevention startups and then divide the results by the type of technology used.
Tool Calls:
  tavily_search (afed72ae4e584a7bb83311ada46bef6a)
 Call ID: afed72ae4e584a7bb83311ada46bef6a
  Args:
    sub_queries: ['wildfire prevention startups']
    topic: general
name='tavily_search' description='Perform searches for each sub-query using the Tavily search tool.' args_schema=<class '__main__.SearchInput'> return_direct=True coroutine=<function tavily_search at 0x12fbe9080>
{'query': 'wildfire prevention startups', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'BurnBot raises $20 million to build technology for wildfire prevention', 'url': 'https://www.cnbc.com/2024/04/02/burnbot-raises-20-million-to-bu

In [ ]:
        # tool descriptions that the model has access to
        # tools = [
        #    {
        #        "name": "tavily_search",
        #        "description": "Connect to a general/news web search engine to gather more information on user's query",
        #        "parameter_definitions": {
        #            "type": {
        #                "description": "type of search to run, 'general', 'news' or both",
        #                "type": "str",
        #                "required": True
        #            }
        #        }
        #    }
        # ]